In [61]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import psycopg2
import numpy as np
DB_HOST = 'localhost'
DB_NAME = 'us'
DB_USER = 'postgres'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn
conn = get_db_connection()
cursor = conn.cursor()
symbol = "UNH"
cursor.execute(f"SELECT trade_date, factor from factors where factor_name = 'option_delta20_using_prev_close' and symbol = '{symbol}' order by trade_date asc")
conn.commit()
date_factor_dict = {i[0]: i[1] for i in cursor.fetchall()}

In [63]:
df = yf.download(symbol, start='2023-12-01')
date_factor_pd = pd.DataFrame([date_factor_dict]).T
date_factor_pd.columns = ['signal']
date_factor_pd['price'] = df['Close'].loc[date_factor_pd.index[0]:date_factor_pd.index[-1]]
date_factor_pd['pct'] = np.log(date_factor_pd['price']/date_factor_pd['price'].shift(1)).shift(0)
# date_factor_pd['strat'] = date_factor_pd.apply(lambda row: row['pct'] if row['signal'] > 1 else -row['pct'], axis=1)
date_factor_pd['strat'] = date_factor_pd.apply(lambda row: row['pct'] if row['signal'] > 1 else 0, axis=1)
date_factor_pd['strat_signal_1_0'] = date_factor_pd['signal'].apply(lambda x: 1 if x > 1 else 0)
# date_factor_pd['strat'].cumsum().plot()


fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles="GS")

date_factor_pd['diff'] = date_factor_pd['strat'].cumsum() - date_factor_pd['pct'].cumsum()
fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['pct'].cumsum(),
    mode='lines',
    name='bm'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['strat'].cumsum() * 0.95,
    mode='lines',
    name='strat'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['diff'],
    mode='lines',
    name='exceed return'
), row=1, col=1)


fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.add_trace(go.Scatter(
    x=date_factor_pd.index,
    y=date_factor_pd['strat_signal_1_0'],
    mode='lines',
    name='Signal'
), row=3, col=1)

fig.update_layout(
    title="GS",
    template="plotly_white",
    height=800,
    width=1200
)


[*********************100%***********************]  1 of 1 completed
